# Gradio Applications

In [2]:
import os 
from dotenv import load_dotenv 
from openai import OpenAI 
import gradio as gr
from IPython.display import Markdown, display

In [3]:
# load the dotenv file 
load_dotenv(override=True)

True

In [4]:
# get the keys 
api_key=os.getenv("OPENAI_API_KEY")
gemini_api_key=os.getenv("GOOGLE_API_KEY")
ollama_api_key=os.getenv("OLLAMA_API_KEY")

# get the base url for gemini and ollama 
ollama_base_url=os.getenv("OLLAMA_BASE_URL")
gemini_base_url=os.getenv("GEMINI_BASE_URL")

In [5]:
# create model instances 
openai=OpenAI()
gemini=OpenAI(base_url=gemini_base_url, api_key=gemini_api_key)
ollama=OpenAI(base_url=ollama_base_url, api_key=ollama_api_key)

# Current Date for LLMs 
are never current system date. The model is trained with information till cut off date. LLMs have this date as current date.

In [13]:
# Let's create system message for ollama models 
system_message = "You are a helpful assistant. And respond to question asked"

def message_ollama(prompt, model="mistral"):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = ollama.chat.completions.create(model=model, messages=messages)
    return response.choices[0].message.content

In [14]:
message_ollama("What is today's date?")

" Today's date, as of my last update, is February 26, 2023. However, I recommend verifying this information because my data might not be current."

In [15]:
message_ollama("What is today's date?", "llama3.1")

'However, I\'m a large language model, I don\'t have real-time access to the current date and time. But I can suggest ways for you to find out the current date.\n\nYou can check your device\'s calendar or clock app, or search online for "current date" to get the latest information. If you want, I can also provide you with a virtual representation of today\'s date as of our conversation, but please note that it will not be 100% accurate since my training data is based on a specific time period and may not reflect the current moment. Would you like me to give you an approximate date?'

# Creating user interface with Gradio 

In [16]:
def shout(text:str):
    print(f"Shout function called with {text}")
    return text.upper()

In [19]:
shout('hello')

Shout function called with hello


'HELLO'

In [20]:
# provide function call back to gradio
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Shout function called with hello


# Sharing Gradio tool with Others
you can share the gradio app in 2 ways 
1. Using deploy() method -> Suggested for longetivity 
2. Using parameter in launch(share=True) -> this will create a temporary instance on hugging face with 7 days retention period. For it to work you should have your notebook opened all the time 
* Important - Some antivirus software or companies LAN may not allow this. 

In [21]:
# gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

### Adding inbrowser settings

In [ ]:
# use parameter inbrowser=True within launch() function to launch the app in browser window 
# gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

### Adding user authentication

In [ ]:
# gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("rohit", "abcd@1234"))

### Forking dark mode

In [22]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Shout function called with hello


### Add example text for users 

In [23]:
message_input = gr.Textbox(label="Your message:", info="Enter a message to be shouted", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=shout,
    title="Shout", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Shout function called with hello
Shout function called with howdy


# Add gradio to ollama call

In [24]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for Mistral", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=message_ollama,
    title="OLLAMA", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy", "Explain 'Attention is all you need' paper in concise manner"], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


### Add streaming feature

In [25]:
def stream_ollama(prompt, model="mistral"):
    messages=[
        {"role": "system", "content": system_message}, 
        {"role": "user", "content": prompt}
    ]

    stream=ollama.chat.completions.create(model=model, messages=messages, stream=True)
    result="" 
    for chunk in stream:
        result+=chunk.choices[0].delta.content or ""
        yield result

In [27]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for Mistral", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_ollama,
    title="OLLAMA", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy", "Explain 'Attention is all you need' paper in concise manner"], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


# Add option for user to select the model for interaction 

In [29]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["mistral", "llama3.1", "phi4-mini", "gemma3"], label="Select model", value="mistral")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_ollama,
    title="LLMs", 
    inputs=[message_input, model_selector],                         # pass model name in the parameter
    outputs=[message_output], 
    examples=[
            ["Explain the Transformer architecture to a layperson", "mistral"],
            ["Explain the Transformer architecture to an aspiring AI engineer", "llama3.1"]
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


# Creating Company brochure - Next level of Week1 Excercise 

In [35]:
from scraper import fetch_website_contents, fetch_website_links
import json 

In [32]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [33]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [36]:
def select_relevant_links(url:str, model:OpenAI=ollama, model_name:str="gemma3"):
    response=model.chat.completions.create(model=model_name, messages=[
        {"role" : "system", "content" : link_system_prompt}, 
        {"role" : "user", "content" : get_links_user_prompt(url)}], response_format={"type" : "json_object"
    })

    result=response.choices[0].message.content
    links=json.loads(result)

    print(f"Found {len(links['links'])} relevant links")

    return links

In [37]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [38]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [39]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [42]:
def stream_brochure(company_name, url, model_name="llama3.1", model:OpenAI=ollama):
    stream=model.chat.completions.create(model=model_name, messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ], stream=True)

    response=""
    # display_handle=display(Markdown(""), display_id=True)

    for chunk in stream:
        response+=chunk.choices[0].delta.content or ""
        # update_display(Markdown(response), display_id=display_handle.display_id)
        yield response 

In [43]:
name_input = gr.Textbox(label="Company name:")
url_input = gr.Textbox(label="Landing page URL including http:// or https://")
model_selector = gr.Dropdown(["mistral", "llama3.1", "phi4-mini", "gemma3"], label="Select model", value="mistral")
message_output = gr.Markdown(label="Response:")

view=gr.Interface(
    fn=stream_brochure, 
    title="Generate Brochure for companies",
    inputs=[name_input, url_input, model_selector],
    outputs=[message_output],
    examples=[
            ["Hugging Face", "https://huggingface.co", "mistral"],
            ["Edward Donner", "https://edwarddonner.com", "llama3.1"]
        ], 
    flagging_mode="never"    

).launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


Found 17 relevant links
